In [12]:
import random, os
import numpy as np
import pandas as pd
import json
from pathlib import Path
from __future__ import annotations
from typing import Dict, Any, List, Tuple, Optional, Callable
from datetime import datetime, timedelta
import requests
from openai import OpenAI
from pydantic import BaseModel, Field
from enum import Enum
# Load .env from either current dir or parent (for OPENAI_API_KEY, etc.)
try:
    from dotenv import load_dotenv
    load_dotenv(dotenv_path=".env"); load_dotenv(dotenv_path="../.env")
except Exception:
    pass

random.seed(7); np.random.seed(7)

In [13]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

#### Load data

In [14]:
DATA_DIR = "/Users/bd11/Documents/Business/Chapter9/AI/RAG/data"
act_df = pd.read_csv(Path(DATA_DIR, 'activities.csv'))
hotel_df = pd.read_csv(Path(DATA_DIR, 'hotels.csv'))
flight_df = pd.read_csv(Path(DATA_DIR, 'flights.csv'))

#### get_weather utility

In [15]:
def _date(s: str) -> datetime:
    """Parse YYYY-MM-DD into a datetime object."""
    return datetime.strptime(s, "%Y-%m-%d")

def _geocode_open_meteo(city: str) -> Optional[Dict[str, float]]:
    """Geocode a city name to lat/lon using Open-Meteo's free geocoder."""
    r = requests.get(
        "https://geocoding-api.open-meteo.com/v1/search",
        params={"name": city, "count": 1},
        timeout=20
    )
    r.raise_for_status()
    res = r.json().get("results") or []
    if not res:
        return None
    x = res[0]
    return {"name": x["name"], "lat": x["latitude"], "lon": x["longitude"]}

def get_daily_forecast(city: str, start: str, end: str) -> Dict[str, Any]:
    """
    7-day window forecast for [start, end] using Open-Meteo forecast endpoint.
    Returns daily tmax/tmin and precipitation probability.
    """
    loc = _geocode_open_meteo(city)
    if not loc:
        return {"mode": "forecast", "city": city, "error": "geocoding_failed"}
    
    r = requests.get(
        "https://api.open-meteo.com/v1/forecast",
        params={
            "latitude": loc["lat"],
            "longitude": loc["lon"],
            "daily": "temperature_2m_max,temperature_2m_min,precipitation_probability_mean,weathercode",
            "start_date": start,
            "end_date": end,
            "timezone": "auto",
        },
        timeout=25
    )
    r.raise_for_status()
    d = r.json().get("daily", {})
    out = []
    for i, day in enumerate(d.get("time", [])):
        out.append({
            "date": day,
            "tmax": d.get("temperature_2m_max", [None])[i],
            "tmin": d.get("temperature_2m_min", [None])[i],
            "pop": d.get("precipitation_probability_mean", [None])[i],  # %
            "wcode": d.get("weathercode", [None])[i],
        })
    return {"mode": "forecast", "city": city, "lat": loc["lat"], "lon": loc["lon"], "days": out}

#### Historical weather functions

In [16]:
def _wmo_to_condition(
    wcode: int | None,
    precip_mm: float | None,
    snow_mm: float | None,
    cloudcover_mean: float | None
) -> str:
    """Map WMO weather code + precip/snow/cloudcover to a coarse label."""
    precip = (precip_mm or 0.0)
    snow = (snow_mm or 0.0)

    # Hard overrides by actual measured precip/snow
    if snow >= 1.0:
        return "snowy"
    if precip >= 1.0:
        return "rainy"

    # WMO code mapping
    if wcode is not None:
        if wcode == 0:
            return "sunny"
        if 1 <= wcode <= 3 or wcode in (45, 48):
            if cloudcover_mean is not None and cloudcover_mean <= 25:
                return "sunny"
            return "cloudy"
        if (51 <= wcode <= 67) or (80 <= wcode <= 82) or (95 <= wcode <= 99):
            return "rainy"
        if (71 <= wcode <= 77) or (85 <= wcode <= 86):
            return "snowy"

    # Last resort: use cloudcover if we have it
    if cloudcover_mean is not None:
        return "sunny" if cloudcover_mean <= 25 else "cloudy"

    return "cloudy"

def _archive_range(lat: float, lon: float, start: str, end: str) -> dict:
    """Fetch daily weather archive for [start, end] at (lat, lon)."""
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start,
        "end_date": end,
        "daily": ",".join([
            "temperature_2m_max",
            "temperature_2m_min",
            "precipitation_sum",
            "snowfall_sum",
            "weathercode",
            "cloudcover_mean",
        ]),
        "timezone": "auto",
    }
    r = requests.get("https://archive-api.open-meteo.com/v1/archive", params=params, timeout=30)
    r.raise_for_status()
    j = r.json()

    d = j.get("daily", {})
    times = d.get("time", []) or []
    tmaxs = d.get("temperature_2m_max", []) or []
    tmins = d.get("temperature_2m_min", []) or []
    psums = d.get("precipitation_sum", []) or []
    ssums = d.get("snowfall_sum", []) or []
    wcodes = d.get("weathercode", []) or []
    clouds = d.get("cloudcover_mean", []) or []

    out = []
    n = len(times)
    for i in range(n):
        date = times[i]
        tmax = tmaxs[i] if i < len(tmaxs) else None
        tmin = tmins[i] if i < len(tmins) else None
        pmm = psums[i] if i < len(psums) else None
        smm = ssums[i] if i < len(ssums) else None
        w = wcodes[i] if i < len(wcodes) else None
        cc = clouds[i] if i < len(clouds) else None

        cond = _wmo_to_condition(
            int(w) if w is not None else None,
            float(pmm) if pmm is not None else None,
            float(smm) if smm is not None else None,
            float(cc) if cc is not None else None,
        )

        out.append({
            "date": date,
            "tmax": tmax,
            "tmin": tmin,
            "precip_mm": pmm,
            "snow_mm": smm,
            "cloudcover_mean": cc,
            "weathercode": w,
            "condition": cond
        })

    return {
        "latitude": j.get("latitude"),
        "longitude": j.get("longitude"),
        "daily": out
    }

def get_historical_weather_stats(city: str, start: str, end: str, years_back: int = 5) -> dict:
    """
    For trips > 7 days away: sample the same date range over the last N years.
    Returns wet/fair ratios + condition counts + average tmin/tmax across samples.
    """
    loc = _geocode_open_meteo(city)
    if not loc:
        return {"mode": "historical", "city": city, "error": "geocoding_failed"}

    s_dt = datetime.strptime(start, "%Y-%m-%d")
    e_dt = datetime.strptime(end, "%Y-%m-%d")
    span_days = (e_dt - s_dt).days + 1
    this_year = datetime.now().year

    # Aggregates
    wet_days = fair_days = 0
    cond_counts = {"sunny": 0, "cloudy": 0, "rainy": 0, "snowy": 0}
    tmax_acc = 0.0
    tmin_acc = 0.0
    temp_samples = 0
    total_samples = 0
    per_year = []

    for y in range(1, years_back + 1):
        year = this_year - y
        s_y = s_dt.replace(year=year)
        e_y = s_y + timedelta(days=span_days - 1)
        block = _archive_range(loc["lat"], loc["lon"], s_y.strftime("%Y-%m-%d"), e_y.strftime("%Y-%m-%d"))
        days = block.get("daily", [])

        year_wet = year_fair = 0
        year_cond = {"sunny": 0, "cloudy": 0, "rainy": 0, "snowy": 0}

        for d in days:
            precip = float(d.get("precip_mm") or 0.0) + float(d.get("snow_mm") or 0.0)
            if precip >= 1.0:
                wet_days += 1
                year_wet += 1
            else:
                fair_days += 1
                year_fair += 1
            total_samples += 1

            cond = d.get("condition", "cloudy")
            if cond in year_cond:
                year_cond[cond] += 1
                cond_counts[cond] += 1

            # temps
            if d.get("tmax") is not None:
                tmax_acc += float(d["tmax"])
                temp_samples += 1
            if d.get("tmin") is not None:
                tmin_acc += float(d["tmin"])

        per_year.append({
            "year": year, "wet": year_wet, "fair": year_fair, "total": year_wet + year_fair,
            "conditions": year_cond
        })

    wet_ratio = (wet_days / total_samples) if total_samples else 0.0
    fair_ratio = (fair_days / total_samples) if total_samples else 0.0
    avg_tmax = (tmax_acc / temp_samples) if temp_samples else None
    avg_tmin = (tmin_acc / temp_samples) if temp_samples else None

    return {
        "mode": "historical",
        "city": city,
        "lat": loc["lat"], "lon": loc["lon"],
        "years_back": years_back,
        "samples": total_samples,
        "summary": {
            "wet_days": wet_days,
            "fair_days": fair_days,
            "wet_ratio": wet_ratio,
            "fair_ratio": fair_ratio,
            "avg_tmax": avg_tmax,
            "avg_tmin": avg_tmin,
            "conditions": cond_counts
        },
        "per_year": per_year
    }

#### embedding_rank from RAG

In [17]:
def embedding_rank(query: str, docs: List[str], k: int = 5) -> List[Tuple[float, str]]:
    """
    Semantic ranking with sentence-transformers (cosine similarity).
    Falls back with a friendly error if package isn't installed.
    """
    try:
        from sentence_transformers import SentenceTransformer, util
    except ImportError as e:
        print("Embeddings unavailable (install sentence-transformers). Using exact match fallback.")
        # Fallback to exact string matching
        matches = []
        for doc in docs:
            if query.lower() in doc.lower():
                matches.append((1.0, doc))
        return matches[:k]

    model = SentenceTransformer("all-MiniLM-L6-v2")
    doc_emb = model.encode(docs, convert_to_tensor=True, normalize_embeddings=True)
    q_emb = model.encode([query], convert_to_tensor=True, normalize_embeddings=True)[0]
    cos = (doc_emb @ q_emb).cpu().numpy()
    order = np.argsort(cos)[::-1][:k]
    return [(float(cos[i]), docs[i]) for i in order]

#### WEATHER-AWARE ACTIVITY CLASSIFICATION

In [18]:
class WeatherCondition(Enum):
    """Weather condition categories for activity planning"""
    SUNNY = "sunny"
    CLOUDY = "cloudy" 
    RAINY = "rainy"
    SNOWY = "snowy"

class ActivityType(Enum):
    """Activity types based on weather suitability"""
    OUTDOOR = "outdoor"
    INDOOR = "indoor"
    FLEXIBLE = "flexible"  # Can be done in any weather

def classify_weather_from_forecast(day_data: Dict[str, Any]) -> WeatherCondition:
    """
    Classify weather condition from forecast data.
    Uses precipitation probability and weather code to determine condition.
    """
    pop = day_data.get("pop", 0) or 0  # precipitation probability
    wcode = day_data.get("wcode")
    
    # High precipitation probability suggests rain
    if pop > 60:
        return WeatherCondition.RAINY
    
    # Use weather code for more precise classification
    if wcode is not None:
        if wcode == 0:  # Clear sky
            return WeatherCondition.SUNNY
        elif 1 <= wcode <= 3:  # Partly cloudy to overcast
            return WeatherCondition.CLOUDY if pop < 30 else WeatherCondition.RAINY
        elif 45 <= wcode <= 48:  # Fog
            return WeatherCondition.CLOUDY
        elif 51 <= wcode <= 67 or 80 <= wcode <= 82 or 95 <= wcode <= 99:  # Rain variants
            return WeatherCondition.RAINY
        elif 71 <= wcode <= 77 or 85 <= wcode <= 86:  # Snow variants
            return WeatherCondition.SNOWY
    
    # Default based on precipitation probability
    if pop < 20:
        return WeatherCondition.SUNNY
    elif pop < 50:
        return WeatherCondition.CLOUDY
    else:
        return WeatherCondition.RAINY

def get_suitable_activities(weather: WeatherCondition, available_activities: pd.DataFrame) -> pd.DataFrame:
    """
    Filter activities based on weather conditions.
    Returns activities suitable for the given weather.
    """
    if weather == WeatherCondition.SUNNY:
        # Prefer outdoor activities on sunny days, but include all
        return available_activities.sort_values(['theme'], ascending=False)
    
    elif weather == WeatherCondition.RAINY or weather == WeatherCondition.SNOWY:
        # Prioritize indoor activities during bad weather
        indoor_activities = available_activities[available_activities['theme'] == 'indoor']
        other_activities = available_activities[available_activities['theme'] != 'indoor']
        return pd.concat([indoor_activities, other_activities]).reset_index(drop=True)
    
    else:  # CLOUDY - good for both indoor and outdoor
        return available_activities.sort_values('rating', ascending=False)

#### ENHANCED WEATHER-AWARE TRIP PLANNER CLASS

In [19]:
class WeatherAwareTripPlanner:
    """
    Enhanced trip planner that uses weather data to optimize activity selection.
    """
    
    def __init__(self, flight_df: pd.DataFrame, hotel_df: pd.DataFrame, activity_df: pd.DataFrame):
        """Initialize the planner with data sources."""
        self.flight_df = flight_df
        self.hotel_df = hotel_df  
        self.activity_df = activity_df
        
    def _determine_weather_strategy(self, start_date: str) -> str:
        """
        Determine whether to use forecast or historical weather data.
        Returns 'forecast' if trip is within 7 days, 'historical' otherwise.
        """
        start_dt = _date(start_date)
        today = datetime.now()
        days_until_trip = (start_dt - today).days
        
        print(f"📅 Trip starts in {days_until_trip} days")
        
        if days_until_trip <= 7:
            return "forecast"
        else:
            return "historical"
    
    def _get_weather_data(self, destination: str, start_date: str, end_date: str) -> Dict[str, Any]:
        """
        Get appropriate weather data based on trip timing.
        """
        strategy = self._determine_weather_strategy(start_date)
        
        print(f"🌤️  Using {strategy} weather strategy for {destination}")
        
        if strategy == "forecast":
            return get_daily_forecast(destination, start_date, end_date)
        else:
            return get_historical_weather_stats(destination, start_date, end_date)
    
    def _analyze_historical_weather(self, weather_data: Dict[str, Any], trip_days: int) -> Dict[str, Any]:
        """
        Analyze historical weather data to plan activity distribution.
        """
        summary = weather_data["summary"]
        conditions = summary["conditions"]
        total_samples = weather_data["samples"]
        
        # Calculate ratios for planning
        outdoor_suitable_days = conditions["sunny"] + conditions["cloudy"]
        indoor_preferred_days = conditions["rainy"] + conditions["snowy"]
        
        # Estimate for trip duration
        outdoor_ratio = outdoor_suitable_days / total_samples if total_samples > 0 else 0.5
        indoor_ratio = indoor_preferred_days / total_samples if total_samples > 0 else 0.5
        
        estimated_outdoor_days = round(outdoor_ratio * trip_days)
        estimated_indoor_days = trip_days - estimated_outdoor_days
        
        return {
            "strategy": "historical",
            "total_days": trip_days,
            "estimated_outdoor_days": estimated_outdoor_days,
            "estimated_indoor_days": estimated_indoor_days,
            "outdoor_ratio": outdoor_ratio,
            "indoor_ratio": indoor_ratio,
            "historical_conditions": conditions,
            "avg_temp_range": f"{summary.get('avg_tmin', 'N/A')}°C - {summary.get('avg_tmax', 'N/A')}°C"
        }
    
    def _analyze_forecast_weather(self, weather_data: Dict[str, Any]) -> Dict[str, Any]:
        """
        Analyze forecast weather data for day-by-day planning.
        """
        daily_conditions = []
        
        for day in weather_data["days"]:
            condition = classify_weather_from_forecast(day)
            daily_conditions.append({
                "date": day["date"],
                "condition": condition.value,
                "temp_max": day.get("tmax"),
                "temp_min": day.get("tmin"),
                "precipitation_prob": day.get("pop", 0)
            })
        
        return {
            "strategy": "forecast",
            "daily_conditions": daily_conditions
        }
    
    def _get_filtered_activities(self, destination: str, themes: List[str]) -> pd.DataFrame:
        """
        Get activities filtered by destination and themes using embedding similarity.
        """
        # Filter by destination first
        dest_activities = self.activity_df[self.activity_df['city'] == destination].copy()
        
        if dest_activities.empty:
            print(f"⚠️  No activities found for {destination}")
            return dest_activities
        
        # Use embedding similarity for theme matching
        close_themes = set()
        available_themes = dest_activities['theme'].unique().tolist()
        
        for theme in themes:
            candidate_themes = embedding_rank(
                query=theme, 
                docs=available_themes,
                k=3
            )
            print(f"🎯 Theme '{theme}' matched to: {[t for s, t in candidate_themes]}")
            close_themes.update([t for s, t in candidate_themes])
        
        # Filter activities by matched themes
        filtered_activities = dest_activities[dest_activities['theme'].isin(close_themes)]
        
        print(f"📍 Found {len(filtered_activities)} activities matching themes in {destination}")
        return filtered_activities
    
    def plan_trip(
        self,
        origin: str,
        destination: str,
        start_date: str,
        end_date: str,
        budget: int,
        themes: Union[str, List[str]]
    ) -> str:
        """
        Create a weather-aware travel plan.
        
        Args:
            origin: Starting city
            destination: Destination city
            start_date: Trip start date (YYYY-MM-DD)
            end_date: Trip end date (YYYY-MM-DD) 
            budget: Total budget in USD
            themes: Activity themes (string or list of strings)
        
        Returns:
            Formatted travel plan as markdown string
        """
        print(f"🚀 Planning weather-aware trip from {origin} to {destination}")
        print(f"📅 Dates: {start_date} to {end_date}")
        print(f"💰 Budget: ${budget}")
        
        # Ensure themes is a list
        if isinstance(themes, str):
            themes = [themes]
        
        # Calculate trip duration
        start_dt = _date(start_date)
        end_dt = _date(end_date)
        trip_days = (end_dt - start_dt).days + 1
        
        # Step 1: Get weather data
        weather_data = self._get_weather_data(destination, start_date, end_date)
        
        if "error" in weather_data:
            print(f"❌ Weather data error: {weather_data['error']}")
            weather_analysis = {"strategy": "fallback", "message": "Weather data unavailable, using general planning"}
        else:
            # Step 2: Analyze weather data
            if weather_data["mode"] == "forecast":
                weather_analysis = self._analyze_forecast_weather(weather_data)
            else:
                weather_analysis = self._analyze_historical_weather(weather_data, trip_days)
        
        # Step 3: Get filtered data for each category
        # Flights
        flight_options = self.flight_df[
            (self.flight_df['origin'] == origin) & 
            (self.flight_df['destination'] == destination)
        ].copy()
        
        # Hotels
        hotel_options = self.hotel_df[self.hotel_df['city'] == destination].copy()
        
        # Activities (using embedding similarity)
        activity_options = self._get_filtered_activities(destination, themes)
        
        # Step 4: Create enhanced prompt with weather context
        system_prompt = """
        You are an expert travel agent with deep knowledge of weather patterns and their impact on travel experiences.
        You create detailed, weather-aware travel plans that maximize enjoyment while staying within budget.
        
        Key principles:
        - Prioritize weather-appropriate activities for each day
        - On sunny/clear days, emphasize outdoor activities 
        - On rainy/snowy days, focus on indoor activities
        - Consider temperature ranges for clothing and activity recommendations
        - Balance cost and quality while respecting the budget constraint
        - Include realistic timing with 2+ hours between flights and activities
        """
        
        # Create weather context for the prompt
        if weather_analysis["strategy"] == "forecast":
            weather_context = "DAILY WEATHER FORECAST:\n"
            for day_info in weather_analysis["daily_conditions"]:
                weather_context += f"- {day_info['date']}: {day_info['condition'].title()}, "
                weather_context += f"{day_info.get('temp_min', 'N/A')}°C - {day_info.get('temp_max', 'N/A')}°C, "
                weather_context += f"{day_info.get('precipitation_prob', 0)}% chance of rain\n"
        
        elif weather_analysis["strategy"] == "historical":
            weather_context = f"""HISTORICAL WEATHER ANALYSIS (based on past {weather_data.get('years_back', 5)} years):
- Expected outdoor-suitable days: {weather_analysis['estimated_outdoor_days']} out of {weather_analysis['total_days']}
- Expected indoor-preferred days: {weather_analysis['estimated_indoor_days']} out of {weather_analysis['total_days']}
- Historical conditions: {weather_analysis['historical_conditions']}
- Average temperature range: {weather_analysis['avg_temp_range']}
- Plan {weather_analysis['estimated_outdoor_days']} days with outdoor focus and {weather_analysis['estimated_indoor_days']} days with indoor focus"""
        
        else:
            weather_context = "Weather data unavailable - plan with general activity balance"
        
        task_prompt = f"""
        Create a comprehensive weather-aware travel plan with the following requirements:

        TRIP DETAILS:
        - Origin: {origin}
        - Destination: {destination}
        - Dates: {start_date} to {end_date} ({trip_days} days)
        - Budget: ${budget}
        - Preferred themes: {themes}

        WEATHER CONTEXT:
        {weather_context}

        WEATHER-AWARE PLANNING GUIDELINES:
        1. For sunny/clear days: Prioritize outdoor activities, walking tours, sightseeing
        2. For rainy/snowy days: Focus on museums, indoor attractions, shopping, dining
        3. For cloudy days: Mix of indoor and outdoor activities
        4. Consider temperature for appropriate activity recommendations
        5. If using historical data, distribute outdoor vs indoor activities according to the estimated ratios
        """
        
        # Data context
        data_context = f"""
        AVAILABLE OPTIONS:

        Flight Options:
        {flight_options.to_csv(sep='\t', index=False) if not flight_options.empty else 'No flights found'}

        Hotel Options:
        {hotel_options.to_csv(sep='\t', index=False) if not hotel_options.empty else 'No hotels found'}

        Activity Options:
        {activity_options.to_csv(sep='\t', index=False) if not activity_options.empty else 'No activities found'}
        """
        
        output_requirements = """
        OUTPUT REQUIREMENTS:
        1. Provide a day-by-day itinerary in markdown format
        2. For each day, include:
           - Weather condition/expectation
           - Recommended activities with timing and cost
           - Weather-appropriate clothing suggestions
           - Backup indoor options for outdoor days
        3. Include flight details with 2+ hour buffers
        4. Use tables for cost breakdowns
        5. Provide total cost summary ensuring it fits within budget
        6. Add weather-specific packing recommendations
        """
        
        # Combine all prompts
        full_prompt = "\n\n".join([task_prompt, data_context, output_requirements])
        
        # Step 5: Generate plan using LLM
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": full_prompt}
        ]
        
        try:
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0.3,  # Lower temperature for more consistent planning
                max_tokens=4000
            )
            
            plan = response.choices[0].message.content.strip()
            
            # Add weather analysis summary at the top
            weather_summary = f"""
# 🌤️ Weather-Aware Trip Plan: {origin} → {destination}

## Weather Analysis Summary
- **Strategy**: {weather_analysis['strategy'].title()}
- **Analysis**: {weather_analysis.get('message', 'Weather data successfully incorporated')}

---

{plan}
            """
            
            return weather_summary
            
        except Exception as e:
            return f"❌ Error generating trip plan: {str(e)}"

#### Example Use Case

In [25]:
def demo_weather_aware_planner():
    """
    Demonstrate the weather-aware trip planner with example scenarios.
    """
    
    # Initialize the planner
    planner = WeatherAwareTripPlanner(flight_df, hotel_df, act_df)
    
    print("=" * 80)
    print("🌤️  WEATHER-AWARE TRIP PLANNER DEMO")
    print("=" * 80)
    
    # Example 1: Near-term trip (uses forecast)
    print("\n🔮 EXAMPLE 1: Near-term trip (uses weather forecast)")
    print("-" * 50)
    
    near_term_plan = planner.plan_trip(
        origin="New York City",
        destination="San Francisco",
        start_date="2025-09-15",  # Within 7 days for demo
        end_date="2025-09-18",
        budget=1200,
        themes=["outdoor", "food"]
    )
    
    print("NEAR-TERM PLAN:")
    print(near_term_plan)
    
    # Example 2: Future trip (uses historical weather)
    print("\n\n📊 EXAMPLE 2: Future trip (uses historical weather data)")
    print("-" * 50)
    
    future_plan = planner.plan_trip(
        origin="New York City",
        destination="Seattle", 
        start_date="2025-12-20",  # Far future for demo
        end_date="2025-12-25",
        budget=1500,
        themes=["culture", "indoor", "food"]
    )
    print("FUTURE TRIP PLAN:")
    print(future_plan)
    
    # Example 3: Test weather classification
    print("\n\n🌦️  EXAMPLE 3: Weather classification testing")
    print("-" * 50)
    
    # Mock forecast data for testing
    test_weather_days = [
        {"date": "2025-09-15", "pop": 10, "wcode": 0, "tmax": 25, "tmin": 18},  # Sunny
        {"date": "2025-09-16", "pop": 70, "wcode": 61, "tmax": 20, "tmin": 15}, # Rainy  
        {"date": "2025-09-17", "pop": 30, "wcode": 2, "tmax": 22, "tmin": 16},  # Cloudy
        {"date": "2025-09-18", "pop": 85, "wcode": 71, "tmax": 5, "tmin": -2},  # Snowy
    ]
    
    print("Weather classification examples:")
    for day in test_weather_days:
        condition = classify_weather_from_forecast(day)
        print(f"  {day['date']}: {condition.value.title()} (POP: {day['pop']}%, Code: {day['wcode']})")
    
    # Example 4: Activity filtering by weather
    print("\n\n🎯 EXAMPLE 4: Weather-appropriate activity selection")
    print("-" * 50)
    
    # Test activity filtering for different weather conditions
    sf_activities = act_df[act_df['city'] == 'San Francisco'].copy()
    
    for weather in [WeatherCondition.SUNNY, WeatherCondition.RAINY]:
        suitable_activities = get_suitable_activities(weather, sf_activities)
        print(f"\n{weather.value.title()} day activities:")
        for _, activity in suitable_activities.head(3).iterrows():
            print(f"  - {activity['name']} ({activity['theme']}) - ${activity['cost_usd']}")

In [26]:
demo_weather_aware_planner()

🌤️  WEATHER-AWARE TRIP PLANNER DEMO

🔮 EXAMPLE 1: Near-term trip (uses weather forecast)
--------------------------------------------------
🚀 Planning weather-aware trip from New York City to San Francisco
📅 Dates: 2025-09-15 to 2025-09-18
💰 Budget: $1200
📅 Trip starts in 0 days
🌤️  Using forecast weather strategy for San Francisco
🎯 Theme 'outdoor' matched to: ['nature', 'sports', 'food']
🎯 Theme 'food' matched to: ['food', 'sports', 'nature']
📍 Found 10 activities matching themes in San Francisco
NEAR-TERM PLAN:

# 🌤️ Weather-Aware Trip Plan: New York City → San Francisco

## Weather Analysis Summary
- **Strategy**: Forecast
- **Analysis**: Weather data successfully incorporated

---

# San Francisco Travel Itinerary

## Overview
- **Origin:** New York City
- **Destination:** San Francisco
- **Dates:** September 15, 2025 - September 18, 2025
- **Budget:** $1200
- **Themes:** Outdoor, Food

## Flight Details
- **Flight:** United Airlines F0199
- **Departure:** September 15, 2025, 08:1